# Import Libraries

In [1]:
# import basic libraries
import os
from os import listdir
import sys
import json
import datetime

# import advance libraries
from xml.etree import ElementTree
import skimage.draw
import cv2
import imgaug

# import mask rcnn libraries
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from mrcnn import visualize

# import matplotlib library
import matplotlib.pyplot as plt

# import numpy libraries
import numpy as np
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from numpy import mean

# import keras libraries
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

C:\Users\Eashan\Anaconda3\envs\mask_rcnn\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
Using TensorFlow backend.


In [2]:
%matplotlib inline

In [ ]:
class HolesDataset(Dataset):
    
    # load_dataset function is used to load the train and test dataset
    def load_dataset(self, dataset_dir, is_train=True):
        
        # we add a class that we need to classify in our case it is Holes
        self.add_class("dataset", 1, "Holes")
        
        # we concatenate the dataset_dir with /images and /annots
        images_dir = dataset_dir + '/images/'
        annotations_dir = dataset_dir + '/annots/'
        
        # is_train will be true if we our training our model and false when we are testing the model
        for filename in listdir(images_dir):
            
            # extract image id
            image_id = filename[:-4] # used to skip last 4 chars which is '.jpg' (class_id.jpg)
            
            # if is_train is True skip all images with id greater than and equal to 420
            # roughly 80% of dataset for training
            if is_train and int(image_id) >= 420 :
                continue
            
            # if is_train is not True skip all images with id less than 420
            if not is_train and int(image_id) < 420:
                continue
            
            # declaring image path and annotations path
            img_path = images_dir + filename
            ann_path = annotations_dir + image_id + '.xml'
            
            # using add_image function we pass image_id, image_path and ann_path so that the current
            # image is added to the dataset for training or testing
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    # function used to extract bouding boxes from annotated files
    def extract_boxes(self, filename):

        
        
        # used to parse the .xml files
        tree = ElementTree.parse(filename)
        
        # to get the root of the xml file
        root = tree.getroot()
        
        # we will append all x, y coordinated in boxes
        # for all instances of an onject
        boxes = list()
        
        # we find all attributes with name bndbox
        # bndbox will exist for each ground truth in image
        for box in root.findall('.//bndbox'):
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            coors = [xmin, ymin, xmax, ymax]
            boxes.append(coors)
        
        # extract width and height of the image
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        
        # return boxes-> list, width-> int and height-> int 
        return boxes, width, height
    
    # this function calls on the extract_boxes method and is used to load a mask for each instance in an image
    # returns a boolean mask with following dimensions width * height * instances
    def load_mask(self, image_id):
        
        # info points to the current image_id
        info = self.image_info[image_id]
        
        # we get the annotation path of image_id which is dataset_dir/annots/image_id.xml
        path = info['annotation']
        
        # we call the extract_boxes method(above) to get bndbox from .xml file
        boxes, w, h = self.extract_boxes(path)
        
        # we create len(boxes) number of masks of height 'h' and width 'w'
        masks = zeros([h, w, len(boxes)], dtype='uint8')
        
        # we append the class_id 1 for Hole in our case to the variable
        class_ids = list()
        
        # we loop over all boxes and generate masks (bndbox mask) and class id for each instance
        # masks will have rectange shape as we have used bndboxes for annotations
        # for example:  if 2.jpg have three objects we will have following masks and class_ids
        # 000000000 000000000 000001110 
        # 000011100 011100000 000001110
        # 000011100 011100000 000001110
        # 000000000 011100000 000000000
        #    1         1          1    <- class_ids
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1
            class_ids.append(self.class_names.index('hole'))
        
        # return masks and class_ids as array
        return masks, asarray(class_ids, dtype='int32')
    
    # this functions takes the image_id and returns the path of the image
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

# Hole configuration class, you can change values of hyper parameters here
class HolesConfig(Config):
    # name of the configuration
    NAME = "Holes_cfg"
    
    # Holes class + background class
    NUM_CLASSES = 1 + 1
    
    # steps per epoch and minimum confidence
    STEPS_PER_EPOCH = 361
    
    # learning rate and momentum
    LEARNING_RATE=0.002
    LEARNING_MOMENTUM = 0.8
    
    # regularization penalty
    WEIGHT_DECAY = 0.0001
    
    # image size is controlled by this parameter
    IMAGE_MIN_DIM = 512
    
    # validation steps
    VALIDATION_STEPS = 50
    
    # number of Region of Interest generated per image
    Train_ROIs_Per_Image = 200
    
    # RPN Acnhor scales and ratios to find ROI
    RPN_ANCHOR_SCALES = (16, 32, 48, 64, 128)
    RPN_ANCHOR_RATIOS = [0.5, 1, 1.5]

# load the train dataset
train_set = HolesDataset()
train_set.load_dataset('holesdataset', is_train=True)
train_set.prepare()

# load the test dataset
test_set = HolesDataset()
test_set.load_dataset('holesdataset', is_train=False)
test_set.prepare()

# prepare config by calling the user defined confifuration class
config = HolesConfig()

# define the model
model = MaskRCNN(mode='training', model_dir='./', config=config)

# load weights mscoco model weights
weights_path = 'mask_rcnn_coco.h5'

# load the model weights
model.load_weights(weights_path, 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])


# start the training of model
# you can change epochs and layers (head or all)
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=5, layers='heads')

In [ ]:
mport imgaug.augmenters as iaa
model.train(train_set,
            test_set,
            learning_rate=config.LEARNING_RATE,
            epochs=15,
            layers='all',
            augmentation = iaa.Sometimes(5/6,iaa.OneOf([
            iaa.Fliplr(1),
            iaa.Flipud(1),
            iaa.Affine(rotate=(-45, 45)),
            iaa.Affine(rotate=(-90, 90)),
            iaa.Affine(scale=(0.5, 1.5))
            ])))

In [ ]:
class PredictionConfig(Config):
    NAME = "hole"
    NUM_CLASSES = 1 + 4
    DETECTION_MIN_CONFIDENCE = 0.85
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1


# evaluate_model is used to calculate mean Average Precision of the model
def evaluate_model(dataset, model, cfg):
    APs = list()
    for image_id in dataset.image_ids:

        image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id, use_mini_mask=False)

        scaled_image = mold_image(image, cfg)

        sample = expand_dims(scaled_image, 0)

        yhat = model.detect(sample, verbose=0)

        r = yhat[0]

        AP, _, _, _ = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])

        APs.append(AP)
    maP = mean(APs)
    return mAP

# TODO: create and prepare train and test dataset
# TODO: load prediction configuration with mode 'inference'
# TODO: define model
# TODO: load model weights (your trained model weights)

# evaluate model on train dataset
train_mAP = evaluate_model(train_set, model, cfg)
print("Train mAP: %.3f" % train_mAP[0])

# evaluate model on test dataset
test_mAP = evaluate_model(test_set, model, cfg)
print("Test mAP: %.3f" % test_mAP[1])

In [ ]:
class PredictionConfig(Config):
    # define the name of the configuration
    NAME = "holes"
    # number of classes (background + holes)
    NUM_CLASSES = 1 + 1
    # number of training steps per epoch
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    
# load prediction configuration
cfg = PredictionConfig()

# define the model
model = MaskRCNN(mode='inference', model_dir='./', config=cfg)

# load model weights
model_path = 'your_trained_weights.h5'

# load image and convert o array
image = load_img("image_path.jpg")
image = img_to_array(image)

# call the detect method on image
results = model.detect([image], verbose=1)

# class_names = ['BG', class_id_1_name, class_id_2_name]
class_names = ['hole']

r = results[0]

visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], class_names,  r['scores'])
model.load_weights(model_path, by_name=True) 